In [35]:
import meshplot as mp
import numpy as np

import xml.etree.ElementTree as ET

def load_color_profile(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    vertices = [float(x) for x in root.find("Tags").find("gamutBoundaryDescType").find("Vertices").find("PCSValues").text.split()]
    vertices = np.reshape(vertices, (-1, 3))
    print(vertices)
    triangles = np.array([[int(x) for x in t.text.split()] for t in root.find("Tags").find("gamutBoundaryDescType").find("Triangles")])
    print(triangles)
    return vertices, triangles

def load_gam_file(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
        vertices = []
        triangles = []
        begin_count = 0
        is_read = False
        for line in lines:
            tokens = line.split()
            if not tokens: 
                continue
            if tokens[0] == "BEGIN_DATA":
                begin_count += 1
                is_read = True
                continue
            elif tokens[0] == "END_DATA":
                is_read = False
                continue
            if begin_count == 1 and is_read:
                vertices.append([float(x) for x in tokens[1:]])
            elif begin_count == 2 and is_read:
                triangles.append([int(x) for x in tokens])
    return np.array(vertices), np.array(triangles)


# https://www.easyrgb.com/en/math.php
XYZ_TRISTIMULUS = {'A':{2: [1.09850, 1.00000, 0.35585], 10: [1.11144, 1.00000, 0.35200]},   #	Incandescent/tungsten
                   'C':{2: [0.98074, 1.00000, 1.18232], 10: [.97285, 1.00000, 1.16145]},    # Old daylight
                   'D50':{2: [0.96422, 1.00000, 0.82521], 10: [0.96720, 1.00000, 0.81427]}, # ICC profile PCS
                   'D55':{2: [0.95682, 1.00000, 0.92149], 10: [0.95799, 1.00000, 0.90926]}, # Mid-morning/daylight
                   'D65':{2: [0.95047, 1.00000, 1.08883], 10: [0.94811, 1.00000, 1.07304]}, # Daylight, sRGB, Adobe-RGB
                   'D75':{2: [0.94972, 1.00000, 1.22638], 10: [0.94416, 1.00000, 1.20641]}, # North sky daylight
                   'F2':{2: [0.99187, 1.00000, 0.67395], 10: [1.03280, 1.00000, 0.69026]},  # Cool white fluorescent
                   'F7':{2: [0.95044, 1.00000, 1.08755], 10: [0.95792, 1.00000, 1.07687]}}  # Daylight fluorescent
                   
                                      



def lab_to_xyz(lab_colors, illuminant='D65', observer=2):
    # Convert Lab to XYZ for multiple colors
    L, a, b = lab_colors[:, 0], lab_colors[:, 1], lab_colors[:, 2]
    y = (L + 16) / 116
    x = a / 500 + y
    z = y - b / 200
    x3, y3, z3 = x ** 3, y ** 3, z ** 3
    delta = 6 / 29
    x = np.where(x > delta, x3, (x - 16 / 116) / 7.787)
    y = np.where(y > delta, y3, (y - 16 / 116) / 7.787)
    z = np.where(z > delta, z3, (z - 16 / 116) / 7.787)
    x *= XYZ_TRISTIMULUS[illuminant][observer][0]
    y *= XYZ_TRISTIMULUS[illuminant][observer][1]
    z *= XYZ_TRISTIMULUS[illuminant][observer][2]
    return np.column_stack([x, y, z])

def xyz_to_rgb(xyz_colors):
    # Convert XYZ to RGB for multiple colors
    m = np.array([[3.2406, -1.5372, -0.4986],
                  [-0.9689, 1.8758, 0.0415],
                  [0.0557, -0.2040, 1.0570]])
    return np.dot(xyz_colors, m.T)

def lab_to_rgb(lab_colors, illuminant='D65', observer=2):
    # Convert Lab to RGB for multiple colors
    xyz_colors = lab_to_xyz(lab_colors)
    rgb_colors = xyz_to_rgb(xyz_colors)
    # Clip RGB values to valid range [0, 1] for multiple colors
    return np.clip(rgb_colors, 0, 1)
        

def main():
    # vertices, triangles = load_color_profile("profiles/sRGB_D65_MAT.xml")
    vertices, triangles = load_gam_file("profiles/AdobeRGB1998.gam")
    colors = lab_to_rgb(vertices)
    mp.plot(vertices, triangles,c=colors, shading={  "flat": True})

if __name__ == "__main__":
    main()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(50.0, -18…

In [10]:
import 

SyntaxError: invalid syntax (2494350585.py, line 1)